In [1]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

In [38]:
df = pd.read_csv('./src/네이버종토방댓글_카카오_15000.csv')
df = df[df['댓글'] != 'error'] # 전처리 에러 제거
df = df.dropna() # 결측치 제거
df['한글댓글'] = df['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ') # 답글 태그 제거
df['한글댓글'] = df['한글댓글'].str.replace('답글:',' ')
df['한글댓글'] = df['한글댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip() # 한글만 남기기
df = df[df['한글댓글'] != ''] # 결측치 제거
df = df.reset_index(drop=True) # 인덱스 리셋
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글
0,2022-05-03,개미들 조련하는법,26,1,0,개미들 조련하는법
1,2022-05-03,6만원,38,1,0,만원
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2,카카오는 끝물입니다 사실상
3,2022-05-03,안티 살발하다,34,2,0,안티 살발하다
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1,이런 호로개미지옥탕을 만드신
...,...,...,...,...,...,...
295849,2017-10-18,어차피 또 폭등할텐데 떨필요있나 기회주면...,795,15,3,어차피 또 폭등할텐데 떨필요있나 기회주면
295850,2017-10-18,오늘 물렸다고 생각하는 사람들은 도대체 ...,961,3,2,오늘 물렸다고 생각하는 사람들은 도대체
295851,2017-10-18,내일도,618,5,1,내일도
295852,2017-10-18,아마존 per는 300?,745,4,1,아마존 는


In [32]:
okt = Okt()

tag_list = ['Noun','Verb','Adjective','VerbPrefix']

tokenized_data = []
for i in range(df.shape[0]):
    tokenized_sentence = okt.pos(df['한글댓글'][i], stem=True) # 토큰화
    tag_checked_sentence = []
    for j in tokenized_sentence:
        x,y = j
        if y in tag_list:
            tag_checked_sentence.append(x)
    tokenized_data.append(tag_checked_sentence)
    if i % 10000 == 0:
        print(f'{i}개 토큰화 완료')

0개 토큰화 완료
10000개 토큰화 완료
20000개 토큰화 완료
30000개 토큰화 완료
40000개 토큰화 완료
50000개 토큰화 완료
60000개 토큰화 완료
70000개 토큰화 완료
80000개 토큰화 완료
90000개 토큰화 완료
100000개 토큰화 완료
110000개 토큰화 완료
120000개 토큰화 완료
130000개 토큰화 완료
140000개 토큰화 완료
150000개 토큰화 완료
160000개 토큰화 완료
170000개 토큰화 완료
180000개 토큰화 완료
190000개 토큰화 완료
200000개 토큰화 완료
210000개 토큰화 완료
220000개 토큰화 완료
230000개 토큰화 완료
240000개 토큰화 완료
250000개 토큰화 완료
260000개 토큰화 완료
270000개 토큰화 완료
280000개 토큰화 완료
290000개 토큰화 완료


In [40]:
df['토큰화댓글'] = tokenized_data
df = df[df['토큰화댓글'].str.len() > 1]
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글,토큰화댓글
0,2022-05-03,개미들 조련하는법,26,1,0,개미들 조련하는법,"[개미, 조련, 하다, 법]"
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2,카카오는 끝물입니다 사실상,"[카카오, 끝물, 이다, 사실]"
3,2022-05-03,안티 살발하다,34,2,0,안티 살발하다,"[안티, 살발]"
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1,이런 호로개미지옥탕을 만드신,"[이렇다, 호로, 개미, 지옥탕, 만들다]"
5,2022-05-03,곡소리 났구나.,66,0,0,곡소리 났구나,"[곡, 소리, 나다]"
...,...,...,...,...,...,...,...
295847,2017-10-18,여기서 털리는 빠가사리들은 나가죽어라,693,6,6,여기서 털리는 빠가사리들은 나가죽어라,"[여기, 털리다, 빠가사리, 나가다, 죽다]"
295848,2017-10-18,카카오 네이버 제휴!,935,2,3,카카오 네이버 제휴,"[카카오, 네이버, 제휴]"
295849,2017-10-18,어차피 또 폭등할텐데 떨필요있나 기회주면...,795,15,3,어차피 또 폭등할텐데 떨필요있나 기회주면,"[어차피, 또, 폭등, 하다, 떨다, 필요, 있다, 회주]"
295850,2017-10-18,오늘 물렸다고 생각하는 사람들은 도대체 ...,961,3,2,오늘 물렸다고 생각하는 사람들은 도대체,"[오늘, 물리다, 생각, 하다, 사람, 도대체]"


In [41]:
df.to_csv("./src/네이버종토방댓글_카카오_15000_태그토큰화.csv", index=False)